# AutoML Chatbot

## Step 1: Import the necessary libraries and load env variables


In [136]:
import pgml
import pandas as pd
from pgml import Database
from psycopg_pool import ConnectionPool
import openai
import numpy as np
import itertools
import os
import glob
import codecs


openai_key = os.getenv("OPENAI_KEY")
conn_str = os.getenv("PGML_CONNECTION_STR")
collection_name = os.getenv("PGML_COLLECTION_NAME")





## Step 2: Load the data

Here we retrieve the contents of the markdown files from the postgresml website and prepare the data to be upserted into the database.

In [124]:

test_suite = pd.read_csv('data/test-suite.csv')

## recursively go through '../../pgml-dashboard/content' and save all markdown files content to df
def read_markdown_files(directory):
    data = []
    base_url = "http://postgresml.com/"

    for dirpath, dirnames, filenames in os.walk(directory):
        for file in glob.glob(os.path.join(dirpath, '*.md')):
            with codecs.open(file, 'r', 'utf-8') as f:
                # Compute the relative file path
                relative_path = os.path.relpath(file, directory)
                # Remove '.md' from the end
                relative_path = relative_path[:-3]
                # Prepend the base URL
                url = base_url + relative_path
                data.append((f.read(), url))

    # Convert the list of tuples into a DataFrame
    df = pd.DataFrame(data, columns=['text', 'url'])

    return df.to_dict('records')
documents = read_markdown_files('../../pgml-dashboard/content')


## Upload data to pgml collection

In [125]:
db = Database(conn_str)
collection = await db.create_or_get_collection(collection_name)
await collection.upsert_documents(documents)


## Create utility functions

Created some utility functions that are used in the grid search.

In [126]:
import json
# import random
import openai

def get_similarity(string1, string2):
    pool = ConnectionPool(conn_str)
    # Return a random floating point number between 0 and 1
    # return random.uniform(0, 1)
    # Establish a database connection
    conn = pool.getconn()
    cur = conn.cursor()
    try:
        # Execute a SQL query that computes the cosine similarity between two text strings
        cur.execute("""
        WITH string1 AS (
            SELECT pgml.embed(
            transformer => 'hkunlp/instructor-xl', 
            text => %s,
            kwargs => '{"instruction": "Represent the answer: "}'
            ) AS embedding
            ),
            string2 AS (
                SELECT pgml.embed(
                transformer => 'hkunlp/instructor-xl', 
                text => %s,
                kwargs => '{"instruction": "Represent the answer: "}'
                ) AS embedding
            )
            SELECT pgml.cosine_similarity(string1.embedding, string2.embedding) AS similarity
            FROM string1, string2;
        """, (string1, string2))
        # Fetch the computed similarity
        similarity = cur.fetchone()
    except Exception as e:
        print(e)
    finally:
        # Close the cursor
        cur.close()
        return similarity

def get_completion(prompt, model_name):
    # Create a chat completion using the OpenAI API
    completion = openai.ChatCompletion.create(
      model=model_name,
      messages=[
        {"role": "system", "content": prompt},    
      ]
    )

    return completion

def get_prediction(context, question, model_name):
    # Format the prompt to be sent to the model
    prompt = f"""You are a helpful assistant that answers users questions about PostgresML. Use the context below, which is delimited by three backticks to help answer the users question.
    context: ```{context}```

    user: {question}

    assistant:

    """

    # Create a chat completion using the OpenAI API
    completion = get_completion(prompt, model_name)

    # Extract the model's answer from the API response
    return completion.choices[0].message.content

def check_accuracy(question, correct_answer, prediction):
    # Format the prompt to be sent to the model
    prompt = f"""You are an expert in determining if the answer to a question is correct. Below is the 'question', 'correct answer', and 'possible answer'. Each is delimited by three backticks. Your job is to determine if the 'possible answer' is correct. You compare it to the 'correct answer' to determine this.

Return your answer as JSON with the key 'accurate' which is a boolean of true of false.

question: ```{question}```

correct answer: ```{correct_answer}```

possible answer: ```{prediction}```

Don't explain your reasoning, just return the json.
lets think through this step by step and determine if possible answer is correct."""

    # Create a chat completion using the OpenAI API
    completion = get_completion(prompt, "gpt-4")

    # Extract the model's answer from the API response and parse the JSON string
    answer = json.loads(completion.choices[0].message.content)

    # Check if 'accurate' is a boolean, if not, raise an error
    if type(answer['accurate']) != bool:
        raise ValueError("answer.accurate must be a boolean")

    # If the prediction is accurate, return 1, otherwise return 0
    return 1 if answer['accurate'] else 0


## Define grid search

In [127]:
splitters = [
    {
        "splitter_name": "recursive_character",
        "splitter_params": {"chunk_size": 1500, "chunk_overlap": 40},
        "name": 5,
    },
    {
        "splitter_name": "recursive_character",
        "splitter_params": {"chunk_size": 2000, "chunk_overlap": 40},
        "name": 6,
    },
    {
        "splitter_name": "recursive_character",
        "splitter_params": {"chunk_size": 2500, "chunk_overlap": 40},
        "name": 7,
    },    
]

embedding_models = [  
    {'model_name': "hkunlp/instructor-xl", 'model_params': {"instruction": "Represent the document for retrieval: "}, 'query_params': {"instruction": "Represent the question for retrieving supporting documents: "}, "name": 3},
    {'model_name': "intfloat/e5-small", 'model_params': {}, 'query_params': {}, "name": 1},
]

completion_models = [
    {'model_name': "gpt-3.5-turbo", "name": 1},
    {'model_name': "gpt-4", "name": 2},
]

grid = {
    "splitters": splitters,
    "models": embedding_models,
    "completion_models": completion_models,
}





## Perform Grid Search

In [128]:
async def generate_search_configurations(collection, grid):
    # Generate the configurations for the search operation
    configurations = []
    for splitter, model, completion_model in itertools.product(grid["splitters"], grid["models"], grid["completion_models"]):
        embedding_model_id = await collection.register_model(
            model_name=model["model_name"],
            model_params= model["model_params"],            
        )
        splitter_id = await collection.register_text_splitter(
            splitter_name=splitter["splitter_name"],
            splitter_params=splitter["splitter_params"],        
        )
        ## append id to model and splitter 
        model['id'] = embedding_model_id
        splitter['id'] = splitter_id        
        await collection.generate_chunks(splitter_id=splitter_id)               
        await collection.generate_embeddings(model_id=embedding_model_id, splitter_id=splitter_id)        
        configurations.append((splitter, model, completion_model))
    return configurations


async def generate_predictions(collection, test_suite_copy, configuration):
    # Generate the predictions for the test suite copy based on the configuration
    splitter, model, completion_model = configuration
    for index, row in test_suite_copy.iterrows():        
        res = await collection.vector_search(
            row['input'],
            top_k=2,
            model_id=model['id'],
            splitter_id=splitter["id"],
            query_params=model.get('query_params', {})
        )        

        # Generate prediction by accumulating chunks and feeding to completion model
        chunks = [r[1] for r in res]
        completion = get_prediction(chunks, row['input'], completion_model['model_name'])
        test_suite_copy.at[index, 'prediction'] = completion

        # Calculate and store similarity measure
        test_suite_copy.at[index, 'prediction_similarity'] = get_similarity(row['output'], completion)

        # Check and store accuracy of prediction
        test_suite_copy.at[index, 'accuracy'] = check_accuracy(row['input'], row['output'], completion)

    # Set ids for the different models used in the test
    test_suite_copy['embedding_model_id'] = model['name']
    test_suite_copy['splitter_id'] = splitter["name"]
    test_suite_copy['completion_model_id'] = completion_model["name"]

    return test_suite_copy


async def grid_search(test_suite, grid):    

    # Initializing connection to database
    db = Database(conn_str)

    # Create or get the collection from database
    collection = await db.create_or_get_collection(collection_name)

    # Generate the configurations to test
    configurations = await generate_search_configurations(collection, grid)

    # Generate predictions for each configuration and store the results
    results = []
    for configuration in configurations:
        test_suite_copy = test_suite.copy()
        prediction = await generate_predictions(collection, test_suite_copy, configuration)
        results.append(prediction)

    return results

grid_search_res_list = await grid_search(test_suite, grid)


## Format grid search results

In [131]:
def format_results(grid_search_res_list):
    # Concatenate list, group by ids, get mean, reset index and sort all in one line
    res_df = pd.concat(grid_search_res_list).groupby(['embedding_model_id', 'splitter_id', 'completion_model_id']).mean().reset_index().sort_values(by=['accuracy'], ascending=False)
    
    # Add a new column called model_id and set it to the index
    res_df['model_id'] = res_df.index.astype(str)

    return res_df

results = format_results(grid_search_res_list)


## Evaulate results and write results to csv file

In [135]:
import json
import pandas as pd

def generate_report_and_save(results, model="gpt-4"):
    results_list = results.to_dict('records')
    # Prepare the prompt
    prompt = f"""
    You are an expert mathmetician and data scientist and machine learning You are tasked with evaluating which model_id has the highest accuracy. The results of the evaluation is below, delimited by three backticks. The results is a list of results, where each item is a model. 'completion_model_id', 'splitter_id', and 'embedding_model_id; are the features in the model. 'prediction_similarity' is the mean of the prediction in the prediction. 'accuracy' is the mean of the accuracy of the prediction. 'accuracy' is the metric you are trying to maximize. prediction_similarity is the similarity of the prediction to the correct answer. One would assume that the higher the prediction_similarity, the higher the accuracy is, but that may not always be the case. Analyze the results and determine which model is the best.

    results: ```{results_list}```

    return your answer as JSON with the keys "winning_model_id" and "report" 

    "winning_model_id" is the model_id of the most accurate model. "report" is a string that explains how you came to your conclusion.
    
    """

    # Create completion
    completion = openai.ChatCompletion.create(
        model=model,
        temperature=0,
        messages=[
            {"role": 'assistant', "content": """{"winning_model_id": "model_id", "report": "After analyzing the results I have determined that"}"""},            
            {"role": 'assistant', "content": """{"winning_model_id": "model_id", "report": "After analyzing the results I have determined that"}"""},            
            {"role": 'assistant', "content": """{"winning_model_id": "model_id", "report": "After analyzing the results I have determined that"}"""},            
            {"role": "system", "content": prompt},    
        ]
    )

    # Extract and process the answer
    answer = json.loads(completion.choices[0].message.content) if isinstance(completion.choices[0].message.content, str) else completion.choices[0].message.content
    winning_model_id = answer['winning_model_id']
    report = answer['report']

    # Update the 'report' column and reorder the columns
    results['report'] = results['model_id'].apply(lambda x: report if x == winning_model_id else '')
    cols_order = ['model_id', 'completion_model_id', 'splitter_id', 'embedding_model_id', 'prediction_similarity', 'accuracy', 'report']
    res_df = results[cols_order]

    # Save results to a csv file
    res_df.to_csv('data/grid-search-results.csv', index=False)


generate_report_and_save(results)